In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from time import sleep
import csv

In [ ]:
driver = webdriver.Chrome()
driver.get("https://www.ad.co.il")
driver.maximize_window()
wait = WebDriverWait(driver, 10000)
relevent_features=["מזגן" ,"חניה","סורגים","מרוהטת","משופצת","מחסן","מעלית",'ממ"ד',"נגישות", "מרפסות"]
dataframe_columns=['City','Address',  'Type' ,'Price','Rooms','Build Size', 'Floor']
main_dataframe=pd.DataFrame(columns=dataframe_columns+relevent_features)
file_path = 'C:/Users/Neomi/Desktop/final project in data science/Data_Base'

for j in range(59 , 100):#going through pages
    driver.get(f'https://www.ad.co.il/nadlanrent?sp275=17413&pageindex={j}')
    try:
        elements = driver.find_elements(By.CLASS_NAME, 'card-block')
    except:
        continue
    end = len(elements) - 3
    time.sleep(7)
    for i in range(end):#going through each home box
        page_data=pd.DataFrame(columns=dataframe_columns+relevent_features)
        print('page ' , j , 'item ' ,i)
        elements[i].click()
        time.sleep(7)
        inside_elements = driver.find_element(By.CLASS_NAME, 'col-xxl-4.col-lg-4.col-md-5')
        City = '0'
        Address = '0'
        Type = '0'
        Price = '0'
        Rooms = '0'
        Build_Size = '0'
        Floor = '0'
        
        #------default icons-----
        balcony = 0
        accessibility = 0
        secure_space = 0
        elevator = 0
        Storage = 0
        improved = 0
        furnished = 0
        bars = 0
        parking = 0
        Air_Conditioner = 0
#-------------------------------------------------getting address 
        address_element = inside_elements.find_element(By.CLASS_NAME, 'card-title')
        Address = address_element.text
#------------------------------------------------------------------

#-----------------------------------------------getting price
        try:
            price_element = inside_elements.find_elements(By.CSS_SELECTOR, 'h2.card-title')[1]
            price=(price_element.text)
            price=price.replace('₪', '')
            price=price.replace(',', '')
            price=price.replace(' ', '')    
            Price = int(price)
        except:
            Price = '0'
#-----------------------------------------------------------------


        try:
            city_check = inside_elements.find_elements(By.CSS_SELECTOR, 'td')[4]
            if "עיר" in city_check.text:
                city_to_data = inside_elements.find_elements(By.CSS_SELECTOR, 'td')[5]
                City = city_to_data.text
        except:
            print("failed city at:",i)
            


        try:
            rooms_check = inside_elements.find_elements(By.CSS_SELECTOR, 'td')[8]
            if "חדרים" in rooms_check.text :
                rooms_to_data = inside_elements.find_elements(By.CSS_SELECTOR , 'td')[9].text
                Rooms = float(rooms_to_data.text)
            else:
                try:
                    rooms_check_2 = inside_elements.find_elements(By.CSS_SELECTOR, 'td')[10]
                    if "חדרים" in rooms_check_2.text :
                        rooms_to_data_2 = inside_elements.find_elements(By.CSS_SELECTOR, 'td')[11].text
                        Rooms = float(rooms_to_data_2)
                except:
                    print("failed inner rooms at:",i)
                    
        except:
            print("failed rooms at:",i)
            


        try:
            types_check = inside_elements.find_elements(By.CSS_SELECTOR, 'td')[0]
            if "פרטי הנכס" in types_check.text:
                types_to_data = inside_elements.find_elements(By.CSS_SELECTOR, 'td')[1]
                Type = types_to_data.text
        except:
            print("error at:",Type)


        floor_check = inside_elements.find_elements(By.CSS_SELECTOR, 'td')[12]
        if "קומה" in floor_check.text:
            floor_to_data = inside_elements.find_elements(By.CSS_SELECTOR, 'td')[13].text
            floor_array = floor_to_data.split()
            if(floor_array[0]=="קרקע"):
                floor_array[0]=0
            Floor = int(floor_array[0])
        else:
            floor_check_2 = inside_elements.find_elements(By.CSS_SELECTOR, 'td')[14]
            if "קומה" in floor_check_2.text:
                floor_to_data_2 = inside_elements.find_elements(By.CSS_SELECTOR, 'td')[15].text
                floor_array = floor_to_data_2.split()
                if(floor_array[0]=="קרקע"):
                    floor_array[0]=0
                Floor = int(floor_array[0])
            
        try:
            build_size_check = inside_elements.find_elements(By.CSS_SELECTOR, 'td')[14]
            if "שטח בנוי" in build_size_check.text:
                build_size_to_data = inside_elements.find_elements(By.CSS_SELECTOR, 'td')[15]
                Build_Size = int(build_size_to_data.text)
            else:
                try:
                    build_size_check_2 = inside_elements.find_elements(By.CSS_SELECTOR, 'td')[16]
                    if "שטח בנוי" in build_size_check_2.text:
                        build_size_to_data_2 = inside_elements.find_elements(By.CSS_SELECTOR, 'td')[17]
                        Build_Size = int(build_size_to_data_2.text)
                except:
                    print("failed build at:",i)
                    
        except:
            print("failed build at:",i)
            Build_Size = '0'
            
        outer_icons=inside_elements.find_element(By.CLASS_NAME, 'card-icons')
        disabled_icons=inside_elements.find_elements(By.CLASS_NAME,"disabled") #contains only disabled icons
        icons=outer_icons.find_elements(By.TAG_NAME, "div")#contains all icons
        for icon in icons:
            if(icon not in disabled_icons): 
                if("מרפסת" in icon.text) :
                     balcony = 1
                if("נגישות" in icon.text) :
                     accessibility = 1
                if('ממ"ד' in icon.text) :
                     secure_space = 1 
                if("מעלית" in icon.text) :
                     elevator = 1 
                if("מחסן" in icon.text) :
                     Storage = 1         
                if("משופצת" in icon.text) :
                     improved = 1 
                if("מרוהטת" in icon.text) :
                     furnished = 1  
                if("סורגים" in icon.text) :
                     bars = 1
                if("חניה" in icon.text) :
                     parking = 1        
                if("מזגן" in icon.text) :
                     air_conditioner = 1        
             
        home_list=[City,Address,Type,Price,Rooms,Build_Size,Floor,air_conditioner,parking,bars,furnished,improved,Storage,elevator,secure_space,accessibility,balcony]
        page_data.loc[len(page_data)] = home_list
        display(page_data)
        main_dataframe=pd.concat([main_dataframe,page_data], ignore_index=True)
        display(main_dataframe)
        
        
        wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="pop-modal"]/div/div/div[1]/div/button'))).click()
        main_dataframe.to_csv(file_path + "page" + str(j) + ".csv", encoding="utf-8-sig")
        
main_dataframe.to_csv(file_path + ".csv", encoding="utf-8-sig")
csv_file.close()
driver.quit()
print(csv_file)